# Vector & Relationships

## Installation

This notebook requires the following dependencies:

In [ ]:
%pip install neo4j-graphrag python-dotenv

## Connecting to Neo4j

The following cell creates an instance of the Neo4j Python Driver that the retrievers require to connect to the database.  The driver is created with environment variables set in your `.env` file.


In [1]:
%load_ext dotenv
%dotenv

from os import getenv

NEO4J_URL = getenv("NEO4J_URI") or "neo4j://localhost:7687"
NEO4J_USERNAME = getenv("NEO4J_USERNAME") or "neo4j"
NEO4J_PASSWORD = getenv("NEO4J_PASSWORD") or "neoneoneo"
NEO4J_DATABASE = getenv("NEO4J_DATABASE") or "neo4j"

from neo4j import GraphDatabase

driver = GraphDatabase.driver(
    NEO4J_URL,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
)

driver.verify_connectivity() # Throws an error if the connection is not successful


## Plain vector search

A vector index already exists called `chunkEmbeddings`.  You can [create your own using the `create_vector_index` function](https://github.com/neo4j/neo4j-graphrag-python?tab=readme-ov-file#creating-a-vector-index) or [populate an existing index using the `upsert_vectors` function](https://github.com/neo4j/neo4j-graphrag-python?tab=readme-ov-file#populating-a-vector-index).

In [13]:
INDEX_NAME = "chunkEmbeddings"

from neo4j_graphrag.embeddings import OpenAIEmbeddings
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.retrievers import VectorRetriever

# Create an Embedder object
embedder = OpenAIEmbeddings()

# Initialize the retriever
retriever = VectorRetriever(
    driver,
    neo4j_database=NEO4J_DATABASE,
    index_name=INDEX_NAME,
    embedder=embedder
)

# Instantiate the LLM
llm = OpenAILLM(model_name="gpt-4o", model_params={"temperature": 0})

The `GraphRAG` class creates a retrieval pipeline that accepts a user input, uses a retriever to fetch the context, and uses an LLM to generate an answer.

In [ ]:
from neo4j_graphrag.generation import GraphRAG

# Instantiate the RAG pipeline
rag = GraphRAG(
    retriever=retriever,
    llm=llm
)

# Query the graph
query = "What are the top risk factors that Apple faces?"

vector_response = rag.search(query_text=query, return_context=True, retriever_config={"top_k": 5})

print(vector_response.answer)

In [ ]:
from neo4j import Result

driver.execute_query("MATCH (c:Company {name: 'APPLE INC'})-[:FACES_RISK]-(r:RiskFactor) RETURN r.name AS risk", result_transformer_=Result.to_df)

### Why "Apple" Queries Can Fail in Vector-Cypher Retrieval

When you ask a question like "What are the risks that Apple faces?" using a vector-Cypher retriever, you may not get the structured or complete answer you expect. Here’s why:

- **How Vector-Cypher Works:**  
  - The retrieval process first performs a semantic search over all text chunks in the graph.
  - It retrieves the `top_k` chunks most similar to your query—regardless of which company (or entity) they are about.
  - The Cypher query then starts from each chunk and traverses the graph for related information.

- **The Problem with Entity-Centric Queries:**  
  - If your query is about "Apple," but there are no chunks whose text is semantically similar to your query and also specifically about Apple, the retriever may return:
    - Chunks about other companies.
    - Chunks that mention "risk" but not "Apple."
    - Generic or boilerplate risk factor text.
  - The Cypher query can only traverse from the retrieved chunk—it cannot "filter" or "redirect" to Apple if the chunk isn’t already about Apple.

- **Key Limitation:**  
  - **The chunk is the anchor.** If your query is about an entity (like Apple), but the chunk retrieval is not entity-aware, you may never reach the correct node or context in the graph.
  - This is especially problematic for broad or entity-centric questions, where you want to aggregate or summarize information about a specific node (e.g., a company) rather than just retrieve semantically similar passages.

Vector-Cypher retrieval is powerful for finding relevant context, but it is fundamentally limited by the chunk-centric approach. For entity-centric questions, you need either:

- Chunks that are explicitly about the entity, or
- A retrieval/query strategy that starts from the entity node itself, not from arbitrary text chunks.

## VectorCypherRetriever Example: Detailed Search with Context

You can use `VectorCypherRetriever` to answer nuanced, relationship-driven questions by combining semantic search with graph traversal.

This example uses the graph to find specific information about asset managers.

### Adding context via relationships

The above pipeline will produce a generic, non-deterministic answer.  Adding relationships to the query will provide a deterministic answer based on the contents of the knowledge graph.  We do this with the `VectorCypherRetriever` class.

In [ ]:
from neo4j_graphrag.retrievers import VectorCypherRetriever
from textwrap import wrap

query = "Who are the asset managers most affected by banking regulations?"

asset_manager_query = """
MATCH (node)-[:FROM_DOCUMENT]-(doc:Document)-[:FILED]-(company:Company)-[:OWNS]-(manager:AssetManager)
RETURN company.name AS company, manager.managerName AS AssetManagerWithSharesInCompany, node.text AS context
"""

vector_cypher_retriever = VectorCypherRetriever(
    driver=driver,
    index_name='chunkEmbeddings',
    embedder=embedder,
    retrieval_query=asset_manager_query
)

rag = GraphRAG(llm=llm, retriever=vector_cypher_retriever)
response = rag.search(
    query,
    retriever_config={"top_k": 5},
    return_context=True
)

wrap(response.answer)


In [ ]:
# View the context used in this query
print("Context:", *response.retriever_result.items, sep="\n\n")

**How this works:**

- **Semantic Search:**  
  The vector retriever finds the top-k text chunks most relevant to your query ("What risks connect major tech companies?").

- **Graph Traversal:**  
  For each retrieved chunk (`node`):
  - Follows the `:FROM_DOCUMENT` and `:FILED` relationships to a company (`c1`).
  - Finds all risk factors (`risk`) that `c1` faces.
  - Finds other companies (`c2`) that also face the same risk factor.
  - Ensures that `c1` and `c2` are different companies.

<!-- ![image.png](attachment:image.png) -->

- **Returns:**  
  - `source_company`: The company from the retrieved chunk.
  - `related_companies`: Companies sharing at least one risk with the source company.
  - `shared_risks`: The names of the risk factors connecting these companies.

- **Why this is powerful:**  
  - Leverages the chunk as the semantic anchor, but then uses graph logic to discover structured, multi-entity relationships.
  - Surfaces both the context (from the chunk) and the broader network of shared risks—something that pure semantic or pure graph search alone would struggle to do as effectively.

This approach is ideal for exploratory questions about relationships in your graph, where you want to start from relevant context but end up with structured, comparative insights.
